In [1]:
import numpy as np
import pandas as pd
import pickle
import json
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, RepeatVector, Masking, TimeDistributed
from tensorflow. keras.utils import plot_model
import tensorflow as tf
import re
from collections import defaultdict
import importlib

In [25]:
data=pd.read_csv(r"Files/image_vectors_with_ids.csv")
data.head()

,Image ID,Image Vector
0,458752,[4.80218083e-02 9.07960236e-01 1.19339478e+00 ...
1,262146,[7.87118018e-01 2.44311482e-01 6.21284246e-02 ...
2,524291,[1.1351358 0.643366 1.2675045 0.10811964 1...
3,393221,[2.51035309e+00 2.11022353e+00 4.24877226e-01 ...
4,393223,[1.42517245e+00 4.25842971e-01 9.63381290e-01 ...


In [3]:
questionsdf = pickle.load(open(r"C:\Users\PIYUSH\Downloads\bert_small.pkl", "rb"))
questionsdf.head()

,image_id,question,question_id,vec
0,458752,What is this photo taken looking through?,458752000,"(((tf.Tensor(-0.1294253, shape=(), dtype=float..."
1,458752,What position is this man playing?,458752001,"(((tf.Tensor(-0.11989767, shape=(), dtype=floa..."
2,458752,What color is the players shirt?,458752002,"(((tf.Tensor(-0.085942656, shape=(), dtype=flo..."
3,458752,Is this man a professional baseball player?,458752003,"(((tf.Tensor(-0.11925976, shape=(), dtype=floa..."
4,262146,What color is the snow?,262146000,"(((tf.Tensor(-0.04078256, shape=(), dtype=floa..."


In [4]:
question_features = questionsdf['vec'].values

In [5]:
question_features[18].shape

TensorShape([1, 12, 768])

In [6]:
question_features[0].shape

TensorShape([1, 10, 768])

In [7]:
for i in range(question_features.shape[0]):
    a=question_features[i].shape[1]
    question_features[i]=tf.reshape(question_features[i],[1*a*768])

In [8]:
question_features[0].shape

TensorShape([7680])

In [9]:
def concat_tensor(question,image):
    t=tf.concat([question,image], 0)
    s=11600-t.shape[0]
    paddings = tf.constant([[0, s,]])
    t=tf.pad(t, paddings, "CONSTANT")
    return t

In [10]:
lens = []
for quest in questionsdf['question']:
    lens.append(len(quest))
question_length = np.array(lens)
question_length

array([41, 34, 32, ..., 37, 26, 59])

In [11]:
question_length.shape

(9935,)

In [12]:
image_ids = []
for x in questionsdf['image_id']:
    if x not in image_ids:
        image_ids.append(x)
image_ids = np.array(image_ids) - 1
sorted(image_ids)

[8,
 24,
 29,
 33,
 35,
 48,
 60,
 63,
 70,
 71,
 76,
 77,
 80,
 88,
 91,
 93,
 108,
 109,
 112,
 126,
 137,
 141,
 143,
 148,
 153,
 164,
 193,
 200,
 246,
 249,
 259,
 306,
 307,
 308,
 311,
 314,
 320,
 321,
 325,
 331,
 367,
 369,
 381,
 383,
 388,
 393,
 403,
 418,
 430,
 435,
 437,
 442,
 449,
 470,
 507,
 509,
 513,
 528,
 530,
 531,
 539,
 541,
 561,
 571,
 574,
 580,
 583,
 594,
 596,
 604,
 611,
 619,
 624,
 628,
 633,
 642,
 649,
 655,
 658,
 672,
 680,
 689,
 713,
 715,
 721,
 722,
 734,
 753,
 761,
 780,
 789,
 794,
 796,
 824,
 827,
 852,
 881,
 896,
 901,
 907,
 908,
 912,
 924,
 926,
 933,
 940,
 954,
 964,
 981,
 983,
 995,
 1005,
 1013,
 1024,
 1035,
 1058,
 1071,
 1083,
 1089,
 1098,
 1101,
 1106,
 1107,
 1110,
 1121,
 1138,
 1154,
 1165,
 1167,
 1182,
 1199,
 1203,
 1215,
 1223,
 1231,
 1263,
 1270,
 1281,
 1294,
 1305,
 1306,
 1307,
 1310,
 1314,
 1329,
 1331,
 1354,
 1359,
 1365,
 1374,
 1380,
 1385,
 1389,
 1391,
 1396,
 1400,
 1402,
 1406,
 1407,
 1430,
 1452,
 

In [23]:
image_ids.shape

(1913,)

In [14]:
file=r"C:\Users\PIYUSH\Desktop\v2_mscoco_train2014_annotations.json"
with open(file,'r') as myfile:
    data=myfile.read()

In [15]:
data1=json.loads(data)['annotations']

In [16]:
def make_vocab_answers(annotations, n_answers):
    """Make dictionary for top n answers and save them into text file."""
    answers = defaultdict(lambda: 0)
    for annotation in annotations:
            for answer in annotation['answers']:
                word = answer['answer']
                if re.search(r"[^\w\s]", word):
                    continue
                answers[word] += 1
                
    answers = sorted(answers, key=answers.get, reverse=True)
    assert('<unk>' not in answers)
    top_answers = ['<unk>'] + answers[:n_answers-1] # '-1' is due to '<unk>'
    
    with open('vocab_answers.txt', 'w') as f:
        f.writelines([w+'\n' for w in top_answers])

    print('Make vocabulary for answers')
    print('The number of total words of answers: %d' % len(answers))
    print('Keep top %d answers into vocab' % n_answers)

In [17]:
make_vocab_answers(data1,500) ## this saves a text file in the same folder

Make vocabulary for answers
The number of total words of answers: 135203
Keep top 500 answers into vocab


In [18]:
question_featuresSENTENCE_SPLIT_REGEX = re.compile(r'(\W+)')


def tokenize(sentence):
    tokens = SENTENCE_SPLIT_REGEX.split(sentence.lower())
    tokens = [t.strip() for t in tokens if len(t.strip()) > 0]
    return tokens
def load_str_list(fname):
    with open(fname) as f:
        lines = f.readlines()
    lines = [l.strip() for l in lines]
    return lines


class VocabDict:

    def __init__(self, vocab_file):
        self.word_list = load_str_list(vocab_file)
        self.word2idx_dict = {w:n_w for n_w, w in enumerate(self.word_list)}
        self.vocab_size = len(self.word_list)
        self.unk2idx = self.word2idx_dict['<unk>'] if '<unk>' in self.word2idx_dict else None
    def idx2word(self, n_w):

        return self.word_list[n_w]

    def word2idx(self, w):
        if w in self.word2idx_dict:
            return self.word2idx_dict[w]
        elif self.unk2idx is not None:
            return self.unk2idx
        else:
            raise ValueError('word %s not in dictionary (while dictionary does not contain <unk>)' % w)

    def tokenize_and_index(self, sentence):
        inds = [self.word2idx(w) for w in tokenize(sentence)]

        return inds
        

In [19]:
ans_vocab = VocabDict('vocab_answers.txt')
ans_vocab = ans_vocab.word2idx_dict


In [20]:
answers = np.array(list(ans_vocab.values()))
answers.shape

(500,)

In [26]:
x_tensor=[]
q_id=[]
for i in range(questionsdf.shape[0]):
    image_features = data[data['Image ID']==questionsdf.iloc[i]['image_id']]['Image Vector'].apply(lambda x: 
                               np.fromstring(
                               x.replace('\n','')
                                .replace('[','')
                                .replace(']','')
                                .replace('  ',' '), sep=' '))
    image=tf.convert_to_tensor(image_features, np.float32)
    question=question_features[i]
    if(question.shape[0]<11000):
        x_tensor.append(concat_tensor(question,image[0]))
        q_id.append(questionsdf.iloc[i]["question_id"])

In [27]:
def unique_answer(question_id):
    for answer in data1:
        if answer['question_id']==question_id:
            return answer['multiple_choice_answer']

In [28]:
for i in range(len(x_tensor)):
    x_tensor[i]=x_tensor[i].numpy()

In [29]:
x_train=np.array(x_tensor)
x_train.shape

(9685, 11600)

In [30]:
label=np.zeros(shape=(len(x_tensor),500))
for i in range(len(q_id)):
    ans=unique_answer(q_id[i])
    try:
        key=ans_vocab[ans]
        label[i][key]=1
    except:
        label[i][0]=1

In [31]:
label=np.array(label)
label.shape

(9685, 500)

In [32]:
x_train.shape

(9685, 11600)

In [33]:
h1=200
h2=200
y=500
model = Sequential()
model.add(Dense(h1, input_dim=x_tensor[0].shape[0], activation='relu'))
model.add(Dense(h2, activation='relu'))
model.add(Dense(y, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [35]:
x=x_train[0:500]
y=label[0:500]
y.shape

(500, 500)

In [36]:
history = model.fit(x,y, epochs=1, batch_size=50)

Train on 500 samples
500/500 [==============================] - 8s 15ms/sample - loss: 4.1657 - accuracy: 0.2900


In [26]:
# test_image_features = np.load("Files/test_img_vectors.npy")
# test_image_features[:5]

In [27]:
# test_questions = pd.read_csv('Files/questions_test.csv')
# test_questions.head()

In [28]:
# test_questions.shape

In [29]:
# lens = []
# for quest in test_questions['question']:
#     lens.append(len(quest))
# test_question_length = np.array(lens)
# test_question_length

In [25]:
def VQA_MODEL():
    image_feature_size          = 512
    word_feature_size           = 300
    max_length_questions        = 30
    number_of_dense_layers      = 3
    number_of_hidden_units      = 1024
    activation_function         = 'tanh'
    dropout_pct                 = 0.5

    # Image model
    model_image = Sequential()
    model_image.add(Reshape((image_feature_size,), input_shape=(image_feature_size,)))

   # Language Model
    model_language = Sequential()
#     model_language.add(LSTM(number_of_hidden_units_LSTM, return_sequences=True,input_shape=(max_length_questions, word_feature_size)))
#     model_language.add(LSTM(number_of_hidden_units_LSTM, return_sequences=True))  
#     model_language.add(LSTM(number_of_hidden_units_LSTM, return_sequences=False))
    #model_language.add(Reshape((word_feature_size,), input_shape=(word_feature_size,)))
    model_language.add()
    model.add(Dense(512))


    # combined model
    x = Concatenate()([model_language.output, model_image.output])

    for _ in range(number_of_dense_layers):
        x = Dense(number_of_hidden_units, kernel_initializer='uniform', activation= activation_function)(x)
        x = Dropout(dropout_pct)(x)
        
    x = Dense(50, activation='softmax')(x)
    
    model = Model(inputs = [model_language.input, model_image.input], outputs=x)
    return model

In [59]:
import tensorflow as tf
paddings = tf.constant([[1, 1,], [2, 2]])
paddings

<tf.Tensor: id=47143, shape=(2, 2), dtype=int32, numpy=
array([[1, 1],
       [2, 2]])>

In [26]:
# import argparse

# def get_arguments():

#     parser = argparse.ArgumentParser()
#     # model
#     parser.add_argument('-model'                  , type=str   , default='simple_mlp')
#     parser.add_argument('-num_hidden_units_mlp'   , type=int   , default=1024)
#     parser.add_argument('-num_hidden_units_lstm'  , type=int   , default=512)
#     parser.add_argument('-num_hidden_layers_mlp'  , type=int   , default=3)
#     parser.add_argument('-num_hidden_layers_lstm' , type=int   , default=1)
#     parser.add_argument('-dropout'                , type=float , default=0.5)
#     parser.add_argument('-activation_1'           , type=str   , default='tanh')
#     parser.add_argument('-activation_2'           , type=str   , default='relu')

#     # training
#     parser.add_argument('-seed'                   , type=int   , default=1337)
#     parser.add_argument('-optimizer'              , type=str   , default='rmsprop')
#     parser.add_argument('-nb_epoch'               , type=int   , default=300)
#     parser.add_argument('-nb_iter'                , type=int   , default=200000)
#     parser.add_argument('-model_save_interval'    , type=int   , default=19)
#     parser.add_argument('-batch_size'             , type=int   , default=128)

    # language features
#     parser.add_argument('-word_vector'            , type=str   , default='glove')
#     parser.add_argument('-word_emb_dim'           , type=int   , default=300)
#     parser.add_argument('-vocabulary_size'        , type=int   , default=12603)
#     parser.add_argument('-max_ques_length'        , type=int   , default=26)
#     parser.add_argument('-data_type'              , type=str   , default='TRAIN')

    # image features
#     parser.add_argument('-img_vec_dim'            , type=int   , default=2048)
#     parser.add_argument('-img_features'           , type=str   , default='resnet')
#     parser.add_argument('-img_normalize'          , type=int   , default=0)

    # evaluations
#     parser.add_argument('-nb_classes'             , type=int   , default=1000)
#     parser.add_argument('-class_activation'       , type=str   , default='softmax')
#     parser.add_argument('-loss'                   , type=str   , default='categorical_crossentropy')
#     parser.add_argument('-save_folder'            , type=str   , default='')

#     # data
#     parser.add_argument('-ans_file'               , type=str   , default='data/val_all_answers_dict.json')
#     parser.add_argument('-input_json'             , type=str   , default='data/data_prepro.json')
#     parser.add_argument('-input_img_h5'           , type=str   , default='data/data_img.h5')
#     parser.add_argument('-input_ques_h5'          , type=str   , default='data/data_prepro.h5')


#     return parser.parse_args()

In [55]:
model = 'simple_mlp'
num_hidden_units_mlp = 1024
num_hidden_units_lstm = 512
num_hidden_layers_mlp = 3
num_hidden_layers_lstm = 1
dropout = 0.5
activation_1 = 'tanh'
activation_2 = 'relu'
seed = 1337
optimizer = 'rmsprop'
nb_epoch = 300
nb_iter = 200000
model_save_interval = 19
batch_size = 128
word_vector = 'glove'
word_emb_dim = 300
vocabulary_size = 12603
max_ques_length = 26
data_type = 'TRAIN'
img_vec_dim = 2048
img_features = 'resnet'
img_normalize = 0
nb_classes = 500
class_activation = 'softmax'
loss = 'categorical_crossentropy'
save_folder = ''

In [56]:
def most_common(lst):
    return max(set(lst), key=lst.count)

In [57]:
def get_train_data():

    train_data = {}
    print('loading image feature...')
    #img_feature = np.load('Files/img_vectors_sample.npy')
    train_data['question'] = question_features 
    train_data['answers'] = answers

    print('Normalizing image feature')
    if img_normalize:
        tem = np.sqrt(np.sum(np.multiply(img_feature, img_feature)))
        img_feature = np.divide(img_feature, np.tile(tem,(1,img_vec_dim)))

    return img_feature, train_data

In [30]:
# def get_data_test(args):
#     dataset = {}
#     test_data = {}
#     # load json file
#     print('loading json file...')
#     with open(args.input_json) as data_file:
#         data = json.load(data_file)
#     for key in data.keys():
#         dataset[key] = data[key]

#     # load image feature
#     print('loading image feature...')
#     img_feature = np.load('img_vectors_sample.npy')
    
#     # load h5 file
#     print('loading h5 file...')
#     with h5py.File(args.input_ques_h5,'r') as hf:
#         # total number of training data is 215375
#         # question is (26, )
#         tem = hf.get('ques_test')
#         test_data['question'] = test_questions
#         # max length is 23
#         tem = hf.get('ques_length_test')
#         test_data['length_q'] = np.array(tem)
#         # total 82460 img
#         # -----1~82460-----
#         tem = hf.get('img_pos_test')
#         # convert into 0~82459
#         test_data['img_list'] = np.array(tem)-1
#         # quiestion id
#         tem = hf.get('question_id_test')
#         test_data['ques_id'] = np.array(tem)
#     # MC_answer_test
#     tem = hf.get('MC_ans_test')
#     test_data['MC_ans_test'] = np.array(tem)

#     print('Normalizing image feature')
#     if img_norm:
#         tem =  np.sqrt(np.sum(np.multiply(img_feature, img_feature)))
#         img_feature = np.divide(img_feature, np.tile(tem,(1,args.img_vec_dim)))


#     # make sure the ans_file is provided
#     nb_data_test = len(test_data[u'question'])
#     val_all_answers_dict = json.load(open(args.ans_file))
#     val_answers = np.zeros(nb_data_test, dtype=np.int32)

#     ans_to_ix = {v: k for k, v in dataset[u'ix_to_ans'].items()}
#     count_of_not_found = 0
#     for i in xrange(nb_data_test):
#         qid = test_data[u'ques_id'][i]
#         try : 
#             val_ans_ix =int(ans_to_ix[most_common(val_all_answers_dict[str(qid)])]) -1
#         except KeyError:
#             count_of_not_found += 1
#             val_ans_ix = 480
#         val_answers[i] = val_ans_ix
#     print("Beware: " + str(count_of_not_found) + " number of val answers are not really correct")

#     return img_feature, test_data

In [38]:
train_img_feature, train_data = get_train_data()
# test_img_feature,  test_data, val_answers = get_test_data(args)

train_X = [train_data[u'question'], train_img_feature]
# train_X = np.reshape(train_X, (train_X.shape[0], 1, train_X.shape[1]))
# train_Y = np_utils.to_categorical(train_data[u'answers'], nb_classes)

# # test_X = [test_data[u'question'], test_img_feature]
# # test_Y = np_utils.to_categorical(val_answers, args.nb_classes)


# # model_name = importlib.import_module("models."+model)

# model = VQA_MODEL()
# model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# model.build(train_img_feature.shape)
# model.summary() # prints model layers with weights

loading image feature...
Normalizing image feature
1913


In [29]:
history = model.fit(train_X, train_Y, batch_size = batch_size, epochs=nb_epoch)
# history = model.fit(train_X, train_Y, batch_size = args.batch_size, nb_epoch=args.nb_epoch, validation_data=(test_X, test_Y))

ValueError: Error when checking input: expected reshape_2_input to have shape (300,) but got array with shape (1,)

In [ ]:
model.summary()

In [ ]:
model.save_weights("./VQA_MODEL_WEIGHTS.hdf5")

In [ ]:
VQA_weights_file_name   = "./VQA_MODEL_WEIGHTS.hdf5"

In [ ]:
def get_VQA_model(VQA_weights_file_name):
    ''' Given the VQA model and its weights, compiles and returns the model '''

    from models.VQA.VQA import VQA_MODEL
    vqa_model = VQA_MODEL()
    vqa_model.load_weights(VQA_weights_file_name)

    vqa_model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
    return vqa_model

In [ ]:
vqa_model = get_VQA_model(VQA_weights_file_name)

In [ ]:
vqa_model.predict([question_features, image_features])